# Synapse data download
- About AMP-AD: https://www.synapse.org/#!Synapse:syn2580853/wiki/409846
- MayoRNAseq https://www.synapse.org/#!Synapse:syn3163039

In [1]:
source("http://depot.sagebase.org/CRAN.R")
pkgInstall("synapseClient")
library(synapseClient)

also installing the dependency ‘rjson’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done

TERMS OF USE NOTICE:
    When using Synapse, remember that the terms and conditions of use require that you:
    1) Attribute data contributors when discussing these data or results from these data.
    2) Not discriminate, identify, or recontact individuals or groups represented by the data.
    3) Use and contribute only data de-identified to HIPAA standards.
    4) Redistribute data only under these same terms of use.



Use your own credentials to log in

In [3]:
synapseLogin(username = "jmehtone", password = "4gxtxt6f", rememberMe = TRUE)

Goodbye.
Welcome jmehtone!


Once logged in, you can download data by synapse ID

In [4]:
geneCounts.Formal <- synGet("syn4650257", downloadLocation = "./")
geneCounts <- read.delim(geneCounts.Formal@filePath, stringsAsFactors = F)
anno.Formal <- synGet("syn3817650", downloadLocation = "./")
anno <- read.delim(anno.Formal@filePath, stringsAsFactors = F)

Let's see how the data objects look like

In [5]:
head(anno)
geneCounts[1:5,1:5]

,ID,Source,Tissue,RIN,Diagnosis,Gender,AgeAtDeath,ApoE,FLOWCELL,PMI
1,11492_TCX,MayoBrainBank_Dickson,TemporalCortex,10,PSP,M,73,33,BC47ADACXX,1
2,6810_TCX,MayoBrainBank_Dickson,TemporalCortex,8,PSP,M,74,33,AC5R6PACXX,1
3,1046_TCX,MayoBrainBank_Dickson,TemporalCortex,9,AD,F,72,33,BC5RNRACXX,2
4,1924_TCX,MayoBrainBank_Dickson,TemporalCortex,5.6,Control,F,90_or_above,33,AC5R6PACXX,2
5,1926_TCX,MayoBrainBank_Dickson,TemporalCortex,7.8,Control,F,88,33,AC44HKACXX,2
6,6913_TCX,MayoBrainBank_Dickson,TemporalCortex,8.3,PSP,F,79,33,AC44HKACXX,2


,ensembl_id,X11344_TCX,X11316_TCX,X11431_TCX,X11341_TCX
1,ENSG00000000003,428,498,652,645
2,ENSG00000000005,5,2,5,8
3,ENSG00000000419,921,574,844,1105
4,ENSG00000000457,435,237,437,608
5,ENSG00000000460,275,212,273,391


## Function to convert Ensembl IDs to gene symbols

Install and load required packages

In [8]:
source("http://bioconductor.org/biocLite.R")
biocLite("biomaRt")
library(biomaRt)

Bioconductor version 3.4 (BiocInstaller 1.24.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.4 (BiocInstaller 1.24.0), R 3.3.1 (2016-06-21).
Installing package(s) ‘biomaRt’
also installing the dependencies ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘S4Vectors’, ‘XML’, ‘AnnotationDbi’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'assertthat', 'BH', 'boot', 'car', 'caret', 'cluster',
  'codetools', 'colorspace', 'crayon', 'curl', 'DBI', 'devtools', 'digest',
  'dplyr', 'evaluate', 'forecast', 'foreign', 'formatR', 'ggplot2', 'git2r',
  'htmltools', 'httpuv', 'httr', 'IRdisplay', 'jsonlite', 'knitr', 'lattice',
  'lme4', 'markdown', 'MASS', 'Matrix', 'memoise', 'mgcv', 'mime', 'nlme',
  'nycflights13', 'openssl', 'pbdZMQ', 'pbkrtest', 'quantreg', 'R6', 'Rcpp',
  'RcppArmadillo', 'RcppEig

This function assumes that ensembl IDs are in rownames of data matrix. Duplicate rows with same gene symbol are removed by leaving the one with the most counts.

In [9]:
ensembl_id_to_gene_symbol <- function(data) {
    ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
    genes <- getBM(attributes = c("ensembl_gene_id", "external_gene_name"),
                  filters = "ensembl_gene_id",
                  values = rownames(data),
                  mart = ensembl)
    data <- data[rownames(data)%in%genes[,1],]
    data <- data[order(rowSums(data), decreasing = T),]
    genes <- genes[match(rownames(data), genes[,1]),]
    dupl <- duplicated(genes[,2])
    data <- data[!dupl,]
    genes <- genes[!dupl,]
    rownames(data) <- genes[,2]
    return(data)
}

From _geneCounts_ move ensembl IDs from 1st column to rownames.

In [10]:
rownames(geneCounts) <- geneCounts[,1]
geneCounts <- geneCounts[,-1]
geneCounts[1:5,1:5]

,X11344_TCX,X11316_TCX,X11431_TCX,X11341_TCX,X11289_TCX
ENSG00000000003,428,498,652,645,336
ENSG00000000005,5,2,5,8,3
ENSG00000000419,921,574,844,1105,543
ENSG00000000457,435,237,437,608,228
ENSG00000000460,275,212,273,391,182


Now convert ensembl IDs to symbols

In [11]:
geneCounts <- ensembl_id_to_gene_symbol(geneCounts)
geneCounts[1:5,1:5]

,X11344_TCX,X11316_TCX,X11431_TCX,X11341_TCX,X11289_TCX
MT-RNR2,665988,2912021,2402275,1392906,3015849
MT-CO1,551625,2346291,1885506,854313,2038025
MT-ND4,278304,929240,1111879,485938,1087501
MT-CO3,222925,727335,862345,334136,1004325
MT-RNR1,179218,445659,510768,505585,858833
